In [6]:
import torch.nn as nn
import torch.distributions as dist
import torch.utils.data
from torch.utils.data import sampler
from torch.distributions import kl
class embed_align(nn.Module):
    def __init__(self, vocab_size1, vocab_size2, emb_dimension):
        super(embed_align, self).__init__()
        self.emb_dimension = emb_dimension

        self.embedding = nn.Embedding(vocab_size1, emb_dimension, padding_idx = 0)
        self.BiLSTM = nn.LSTM(emb_dimension, emb_dimension, bidirectional=True, batch_first=True)
        
        self.affine1_mu = nn.Linear(emb_dimension, emb_dimension)
        self.affine2_mu = nn.Linear(emb_dimension, emb_dimension)
        
        self.affine1_sig = nn.Linear(emb_dimension, emb_dimension)
        self.affine2_sig = nn.Linear(emb_dimension, emb_dimension)
        
        self.affine1_L1 = nn.Linear(emb_dimension, emb_dimension)
        self.affine2_L1 = nn.Linear(emb_dimension, vocab_size1)
        self.affine1_L2 = nn.Linear(emb_dimension, emb_dimension)
        self.affine2_L2 = nn.Linear(emb_dimension, vocab_size2)
        
        self.relu = nn.ReLU()
        self.softplus = nn.Softplus()
        self.log_softmax = nn.Softmax(dim=0)
        
    def forward(self, sentence1, sentence2, use_cuda=False):
        # sentence1 & sentence2 are (batches of) list of all ints in a sentence
        # encoder
        sen1_emb = self.embedding(sentence1)
        if len(sen1_emb.shape) == 2: # not a batch
            sen1_emb = sen1_emb.unsqueeze(0)
        h, _ = self.BiLSTM(sen1_emb)
        h1, h2 = torch.split(h, split_size_or_sections=self.emb_dimension, dim =2)
        h = h1 + h2
        mu = self.affine2_mu(self.relu(self.affine1_mu(h)))
        sig = self.relu(self.affine2_sig(self.relu(self.affine1_sig(h))))
        
        sample_norm = dist.multivariate_normal.MultivariateNormal(torch.zeros(self.emb_dimension), torch.eye(self.emb_dimension))
        e = sample_norm.sample()
        if use_cuda:
            z = mu + e.cuda() * sig
        else:
            z = mu + e * sig
    
        # likelihood language 1
        dist_1 = self.log_softmax(self.affine2_L1(self.relu(self.affine1_L1(z))))
        # sum over batch
        sum_1 = torch.sum(dist_1, dim=0)
        likelihood_1 = torch.mean(sum_1, dim=1)
        total_likelihood1 = 0
        sen_len = 0
        for i, likelihood in enumerate(likelihood_1):
            # no batches:
            if len(sentence1) == longest1:
                if sentence1[i].item() == 0:
                    continue
                total_likelihood1 += likelihood
                sen_len +=1
            else:
                for j in range(len(sentence1)):
                    if sentence1[j][i].item() == 0:
                        continue
                    total_likelihood1 += likelihood
                sen_len +=1
        likelihood1 = total_likelihood1/sen_len
        
        # likelihood language 2
        dist_2 = self.log_softmax(self.affine2_L2(self.relu(self.affine1_L2(z))))
        sum_2 = torch.sum(dist_2, dim=0)
        likelihood_2 = torch.mean(sum_2, dim=1)
        total_likelihood2 = 0
        sen_len = 0
        for i, likelihood in enumerate(likelihood_2):
            # no batches:
            if len(sentence1) == longest1:
                if sentence1[i].item() == 0:
                    continue
                total_likelihood2 += likelihood
                sen_len +=1
            else:
                for j in range(len(sentence1)):
                    if sentence1[j][i].item() == 0:
                        continue
                    total_likelihood2 += likelihood
                sen_len +=1
        likelihood2 = total_likelihood2/sen_len
        
        # KL
        # to prevent log returning infinity
        sig = sig+1e-8
        KL =  -0.5 * torch.sum(1 + torch.log(sig) - mu.pow(2) - sig)
        return - ((likelihood1 + likelihood2) - KL)

In [7]:
# read in all needed data

def get_test_data(filename):
    test_data = []
    with open(filename) as f:
        for line in f:
            line = line.split('\t')
            test_word = line[0]
            sentence_id = line[1]
            test_data.append([test_word, sentence_id])
    return(test_data)

def get_candidates(filename):
    with open(filename) as f:
        candidate_dict = {}
        for line in f:
            line = line.split('::')
            word = line[0]
            candidates = line[1].split(';')
            candidate_dict[word] = candidates
        return candidate_dict

In [8]:
import pickle
import torch

test_data = get_test_data('lst/lst_test.preprocessed')

candidates = get_candidates('lst/lst.gold.candidates')

with open('w2i_en_embedalign.pkl', 'rb') as f:
    w2i_en = pickle.load(f)

with open('i2w_en_embedalign.pkl', 'rb') as f:
    i2w_en = pickle.load(f)
    
with open('w2i_fr_embedalign.pkl', 'rb') as f:
    w2i_fr = pickle.load(f)

with open('i2w_fr_embedalign.pkl', 'rb') as f:
    i2w_fr = pickle.load(f)


In [9]:
import string

id_to_sentence = {}

translator = str.maketrans('', '', string.punctuation)

with open('lst/lst_test.preprocessed') as f:
    for line in f:
        line = line.split('\t')
        test_word = line[0]
        sentence_id = line[1]
        sentence = line[3].translate(translator).lower().split()
        #print(sentence)
        id_to_sentence[int(sentence_id)] = sentence

In [10]:
id_to_sentence[1105]

['although',
 'the',
 'mah',
 'look',
 'family',
 'became',
 'a',
 'strong',
 'part',
 'of',
 'community',
 'life',
 'in',
 'the',
 'king',
 'valley',
 'they',
 'had',
 'to',
 'suffer',
 'the',
 'occasional',
 'racist',
 'comments',
 'which',
 'must',
 'have',
 'made',
 'life',
 'unpleasant',
 'at',
 'times']

In [11]:
use_cuda = torch.cuda.is_available()

if use_cuda: # load model with gpu (as it was trained)
    embedalign_model = torch.load('embed_align.pt')
else: # convert to cpu:
    embedalign_model = torch.load('embed_align.pt',  map_location='cpu')

In [12]:
orig_test_data = test_data[:]
for [word, sentence_id] in test_data[:]:
    word_nopos = word[:-2]
    if word_nopos not in w2i_en:
        test_data.remove([word, sentence_id])
    for candidate in candidates[word][:]:
        if candidate not in w2i_en:
            candidates[word].remove(candidate)

In [13]:
word_is = [w2i_en[word[:-2]] for word, _ in test_data]
word_is = torch.LongTensor(word_is)


can_is = {}
for [word, _] in test_data[:]:
    can_i = [w2i_en[can] for can in candidates[word]]
    word_i = w2i_en[word[:-2]]
    can_is[word_i] = torch.LongTensor(can_i)

In [14]:
sentence_is = {}
for [word, sentence_id] in test_data[:]:
    
    sen_i = [w2i_en[sen] for sen in id_to_sentence[int(sentence_id)] if sen in w2i_en.keys()]
    #print(sentence_id,id_to_sentence[int(sentence_id)], sen_i)
    sentence_is[int(sentence_id)] = torch.LongTensor(sen_i)

In [15]:
def get_ranking(word, candidates, sentence, word_pos):
    mu, sigma = embedalign_step(embedalign_model, sentence)

    can_sims=[]
    for can in candidates:
        candidate_sentence = sentence
        candidate_sentence[word_pos] = can
        
        mu_p, sigma_p = embedalign_step(embedalign_model, candidate_sentence)
                                       
        score =  torch.log(sigma_p + 1e-8) - torch.log(sigma+1e-8) + (sigma**2 + (mu - mu_p)**2) / (2*sigma_p**2) - 0.5
        score = score.sum().data.item()
        can_sims.append([can, score])
            
    return(can_sims)

In [16]:
def embedalign_step(model, sentence):

    # Get embeddings for words in sentence
    sen_emb = model.embedding(sentence)

    # - Encoder -
    if len(sen_emb.shape) == 2: # not a batch
        sen_emb = sen_emb.unsqueeze(0)
    h, _ = model.BiLSTM(sen_emb)
        
    h1, h2 = torch.split(h, split_size_or_sections=model.emb_dimension, dim =2)
    h = h1 + h2
    
    
    mu = model.affine2_mu(model.relu(model.affine1_mu(h)))
    sigma = model.relu(model.affine2_sig(model.relu(model.affine1_sig(h))))

    return mu, sigma

In [18]:
results = {}
for i, [word, sentence_id] in enumerate(test_data):
    word_i = word_is[i]
    cans = can_is[word_i.item()]
    
    sentence_ids = sentence_is[int(sentence_id)]
    
    word_pos = -1
    for e, el in enumerate(sentence_ids):
        if el == word_i:
            word_pos = e
        
    
    if (len(cans)>0):
        rank = get_ranking(word_i, cans, sentence_ids, word_pos)
        words_scores = []
        for [candidate, score] in rank:
            can_word = i2w_en[int(candidate)]
            score = score
            words_scores.append([can_word, score])
        results[word] = words_scores

In [20]:
with open('embedalign_predictions', 'w') as f:
    for [word, sentence_id] in orig_test_data: # lst_gap needs all words, also once we do not have data for
        f.write('#RANKED\t')
        f.write(word + ' ')
        f.write(sentence_id)
        if word in results:
            for [candidate, score] in results[word]:
                f.write('\t' + candidate + ' ' + str(score))
        f.write('\n')

In [21]:
%run lst/lst_gap.py lst/lst_test.gold bayesian_skipgram_predictions bayesian_skipgram_out no-mwe


MEAN_GAP	0.06498703223281743

